# **Exploring Data**
This notebook is an exploratory area for understanding my data. 

# Setup
The cells below will help to set up the rest of the notebook. 

I'll start by configuring the kernel that's running this notebook:

In [1]:
# Change the cwd
%cd ..

# Enable the autoreload module
%load_ext autoreload
%autoreload 2

# Load the environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

/Users/thubbard/Documents/personal/programming/pax-pal-2025/experiments


True

Next, I'm going to import the necessary modules:

In [2]:
# General imports
import os

# Third-party imports
import pandas as pd

# Loading Data
Below, I'll load in the data:

In [5]:
games_data_df = pd.read_json("data/final_enriched_games_data.json")

/var/folders/p0/6dyp2b_d72z_d772nvm0y6tc0000gn/T/ipykernel_10022/2821581174.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  games_data_df = pd.read_json("data/final_enriched_games_data.json")
/var/folders/p0/6dyp2b_d72z_d772nvm0y6tc0000gn/T/ipykernel_10022/2821581174.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  games_data_df = pd.read_json("data/final_enriched_games_data.json")
/var/folders/p0/6dyp2b_d72z_d772nvm0y6tc0000gn/T/ipykernel_10022/2821581174.py:1: FutureWarnin

# Exploring Data


### Genres
What are the most common genres / tags?

In [13]:
games_data_df["genres_and_tags"].explode().value_counts().head(20)

genres_and_tags
Indie             85
Adventure         68
Action            55
RPG               31
Simulation        27
Puzzle            22
Casual            21
Strategy          14
Platformer        13
Horror            13
Platform          13
Arcade            12
Roguelite         10
Shooter            9
Early Access       8
Hack and Slash     8
Side Scroller      8
Multiplayer        7
Co-op              6
Singleplayer       6
Name: count, dtype: int64

### Platforms
What platforms are available?

In [16]:
games_data_df["platforms"].explode().value_counts().head(10)

platforms
PC                 104
Nintendo Switch     29
PlayStation 5       20
Xbox One            20
PlayStation 4       15
Tabletop            15
Xbox Series X|S     13
Xbox Series X/S     10
Windows              8
Linux                7
Name: count, dtype: int64

### Miscellaneous

In [14]:
games_data_df.sample(5)

,id,name,snappy_summary,description_texts,platforms,developer,exhibitor,booth_number,header_image_url,steam_link,genres_and_tags,media,released,release_time,links
138,301bd9989ba1,Here Lies,Cooperative RPG mystery game inspired by class...,"[{'source': 'pax_website', 'text': 'In Here Li...",[Tabletop],DVC Games LLC,DVC Games LLC,11112,https://s3.amazonaws.com/app.growtix.com/media...,None,"[RPG, Mystery, Cooperative]",[],0.0,None,"[{'title': 'Google Search for Game', 'url': 'h..."
52,00aadf739c16,Paper Animal Adventure,Cute roguelike adventure with procedurally gen...,"[{'source': 'pax_app', 'text': 'Paper Animal A...","[PC, Nintendo Switch, PlayStation 4, Xbox One,...","Top Hat Studios, Inc.","Top Hat Studios, Inc.",16097,https://s3.amazonaws.com/app.growtix.com/media...,https://store.steampowered.com/app/1982120/Pap...,"[RPG, Mystery Dungeon, Adventure, Dungeon Craw...","[{'type': 'image', 'source': 'steam', 'url': '...",0.0,To be announced,"[{'title': 'Google Search for Game', 'url': 'h..."
153,589f1ad30e3a,Stay: Forever Home,Virtual pet simulation fostering emotional bon...,"[{'source': 'pax_website', 'text': 'Stay: Fore...","[Meta Quest 2, Meta Quest 3, Meta Quest 3S]",Boston Indie Game Developers,Boston Indie Game Developers,18105,https://s3.amazonaws.com/app.growtix.com/media...,None,"[Simulation, Virtual Pet]",[],1.0,"April 24, 2025","[{'title': 'Google Search for Game', 'url': 'h..."
33,4b73279ef330,Iron Meat,Retro-inspired run-and-gun shooter battling in...,"[{'source': 'pax_app', 'text': 'Iron Meat is a...","[PlayStation 5, PlayStation 4, Xbox Series X/S...",Retroware,Retroware,15031,https://s3.amazonaws.com/app.growtix.com/media...,https://store.steampowered.com/app/1157740/Iro...,"[Platform, Side Scroller, Shoot 'Em Up, Shoote...","[{'type': 'image', 'source': 'steam', 'url': '...",1.0,"September 26, 2024","[{'title': 'Google Search for Game', 'url': 'h..."
38,868c3a42a2da,The Zombie Slayers,Colorful top-down shooter blending zombie batt...,"[{'source': 'pax_app', 'text': 'The Zombie Sla...",[PC],indie.io,indie.io,18114,https://s3.amazonaws.com/app.growtix.com/media...,https://store.steampowered.com/app/2932990/The...,"[Multiplayer, Casual, Adventure, Indie, Top-Do...","[{'type': 'image', 'source': 'steam', 'url': '...",0.0,Coming soon,"[{'title': 'Google Search for Game', 'url': 'h..."
